In [1]:
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.utils import np_utils
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import tree
from tensorflow.keras.preprocessing import sequence
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
from google.colab import files
data_to_load = files.upload()
import io
dataset = pd.read_csv(io.BytesIO(data_to_load['amharic.csv']))

Saving amharic.csv to amharic.csv


In [5]:
dataset.drop('Unnamed: 2', inplace=True, axis=1)

In [6]:
dataset.head()

,labels,tweet
0,Free,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,Free\n,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,Free \n,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,Free\n,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,Hate\n,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...


In [7]:
tweet=dataset.tweet
labels=dataset.labels

In [8]:


dataset['labels']=[re.sub(r"\n", "", str(label)) for label in labels]

     


In [10]:
dataset.head()

,labels,tweet
0,Free,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,Free,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,Free,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,Free,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,Hate,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...


In [9]:





# drop function which is used in removing or deleting rows or columns from the CSV files
#dataset.drop('Unnamed: 2', inplace=True, axis=1)

# display
#print("\nCSV Data after deleting the column 'Unnamed: 2':\n")
#print(dataset)


KeyError: ignored

In [11]:
words = dataset['tweet']
finalWords = []
for word in words:
    
    
    #remove any non-alphanumeric character
    word = re.sub(r'\W', ' ', str(word))
    
    #remove any digit
    word = re.sub("(\\d)+"," ",word)
    
    #remove punctuation 
    word = re.sub(",#'!:;"," ",word)
    
    #remove any latin characters 
    word = re.sub(r'[a-zA-Z]+', '', word)

    word = re.sub('\s+', ' ', word)  # remove extra whitespace
    finalWords.append(word)


In [12]:

tokenizedList = []
for lineWord in finalWords:
    separatedWords = lineWord.split(' ')
    word = [word for word in separatedWords if   word.strip() != '']
    tokenizedList.append(word)

In [13]:
# list amharic stop words
stopwords=['ነው', 'እና', 'እኔ', 'የእኔ','እኔ','ራሴ','እኛ','የእኛ','እኛ','ራሳችን','አንቺ' ,'የእርስዎ' ,'እራስዎ','እራሳችሁን','የእሱ',
           'እሱ','እ','ራሱ','እሷ' 'ራሷን','እሱ','በራሱ','እነሱ','የእነሱ','እራሳቸው','ምንድን','የት','ማን','ይሄ','ያንን','እነዚህ','እነዚያ'
           ,'ነው','እነዚህ','ናቸው','ነበር','ነበሩ','መሆን','ሆኗል','መሆን','አለ','ነበሩ','አለው','መስራት','ያደርገዋል','አደረጉት',
           'ማድረግ','እና','ግን','ከሆነ','ወይም','ምክንያቱም','እንደ','እስከ','ገና','የ','ለ','ከ','ጋር','ስለ','መካከል','ውስጥ',
           'ከዚህ','በፊት','በኋላ','ከላይ','ከታች','ወደ','ከ','ወደ','ላይ','ታች','ውስጥ','ውጭ','በላይ','በታች','በድጋሚ','እናስ','እናም']

           
cleanedWordsList= []
for lineWords in tokenizedList:
    tempList = []
    for word in lineWords:
        if word.strip() not in stopwords:
            tempList.append(word)
    cleanedWordsList.append(tempList)

In [14]:
# for the time being this is fine but convert it to something good but not this at all

normalizedSenteces = []
for linesWords in cleanedWordsList:
    tempList = []
    for word in linesWords:
        
        word = re.sub("ሐ","ሀ",word)
        word = re.sub("ሑ","ሁ",word)
        word = re.sub("ሒ","ሂ",word)
        word = re.sub("ሓ","ሃ",word)
        word = re.sub("ሔ","ሄ",word)
        word = re.sub("ሕ","ህ",word)
        word = re.sub("ሖ","ሆ",word)
        
        word = re.sub("ኀ","ሀ",word)
        word = re.sub("ኁ","ሁ",word)
        word = re.sub("ኂ","ሂ",word)
        word = re.sub("ኃ","ሃ",word)
        word = re.sub("ኄ","ሄ",word)
        word = re.sub("ኅ","ህ",word)
        word = re.sub("ኆ","ሆ",word)
        
        word = re.sub("ሠ","ሰ",word)
        word = re.sub("ሡ","ሱ",word)
        word = re.sub("ሢ","ሲ",word)
        word = re.sub("ሣ","ሳ",word)
        word = re.sub("ሤ","ሴ",word)
        word = re.sub("ሥ","ስ",word)
        word = re.sub("ሦ","ሶ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ጸ","ፀ",word)
        word = re.sub("ጹ","ፁ",word)
        word = re.sub("ጺ","ፂ",word)
        word = re.sub("ጻ","ፃ",word)
        word = re.sub("ጼ","ፄ",word)
        word = re.sub("ጽ","ፅ",word)
        word = re.sub("ጾ","ፆ",word)
        
        tempList.append(word)
    normalizedSenteces.append(tempList)

In [15]:
# # Split into training and testing data
x = dataset['tweet']
y = dataset['labels']

In [16]:


x = np.array(dataset["tweet"])
y = np.array(dataset["labels"])
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(x.astype('U')) 
#x = cv.fit_transform(x.apply(lambda x: np.str_(x)))
#X = cv.fit_transform(x)# Fit the Data
#Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


     


In [17]:


print("Training set size:", X_train)
     


Training set size:   (0, 6760)	1
  (0, 789)	1
  (0, 104535)	1
  (0, 24678)	1
  (0, 80280)	1
  (0, 71167)	1
  (0, 64001)	1
  (0, 3002)	1
  (0, 58427)	1
  (0, 20873)	1
  (0, 96863)	1
  (0, 22357)	1
  (0, 47427)	1
  (0, 105964)	1
  (0, 107603)	1
  (0, 96831)	1
  (0, 67851)	1
  (0, 26496)	1
  (0, 1013)	1
  (0, 48833)	1
  (0, 57408)	1
  (0, 68095)	1
  (0, 48497)	1
  (0, 56772)	1
  (0, 30216)	1
  :	:
  (15747, 38570)	1
  (15747, 31653)	1
  (15747, 58117)	1
  (15747, 53397)	1
  (15747, 82862)	1
  (15748, 87040)	1
  (15748, 78976)	1
  (15748, 62682)	1
  (15748, 34421)	1
  (15748, 46232)	1
  (15748, 41144)	1
  (15748, 48107)	1
  (15749, 10680)	1
  (15749, 48709)	1
  (15749, 102085)	1
  (15749, 101885)	1
  (15749, 23582)	1
  (15749, 20873)	1
  (15749, 28209)	1
  (15749, 39096)	1
  (15749, 18222)	1
  (15749, 1648)	1
  (15749, 109231)	1
  (15749, 57063)	1
  (15749, 111349)	1


In [18]:
print("Testing set size:",X_test)

Testing set size:   (0, 44755)	2
  (0, 6760)	1
  (0, 105895)	1
  (0, 15901)	1
  (0, 100569)	1
  (0, 111682)	1
  (0, 111623)	1
  (0, 99878)	1
  (0, 45440)	1
  (0, 17976)	1
  (0, 86487)	1
  (0, 35163)	1
  (0, 16818)	1
  (0, 56137)	1
  (0, 13795)	1
  (0, 31674)	1
  (0, 92475)	1
  (0, 55212)	1
  (0, 66744)	1
  (0, 108137)	1
  (0, 39399)	1
  (1, 44755)	1
  (1, 71642)	1
  (1, 49047)	1
  (1, 92082)	1
  :	:
  (8997, 1749)	1
  (8997, 96512)	1
  (8997, 24678)	1
  (8997, 70782)	1
  (8997, 16457)	1
  (8997, 33332)	1
  (8997, 48853)	1
  (8997, 97535)	1
  (8997, 47081)	1
  (8998, 44755)	1
  (8998, 13451)	1
  (8998, 14945)	1
  (8998, 77871)	1
  (8998, 73602)	1
  (8998, 50973)	1
  (8998, 32202)	1
  (8998, 106057)	1
  (8999, 82912)	1
  (8999, 25209)	1
  (8999, 6484)	1
  (8999, 23790)	1
  (8999, 45440)	1
  (8999, 7076)	1
  (8999, 24340)	1
  (8999, 103411)	1


In [19]:


print("Training set size:", X_val)
     


Training set size:   (0, 7968)	1
  (0, 26376)	1
  (0, 62682)	1
  (0, 45870)	1
  (0, 106301)	1
  (0, 106163)	1
  (0, 4257)	1
  (0, 5238)	1
  (0, 55133)	1
  (0, 42831)	1
  (1, 46489)	1
  (1, 44755)	1
  (1, 65655)	1
  (1, 46495)	1
  (1, 22266)	1
  (1, 67493)	1
  (1, 43945)	1
  (1, 111158)	1
  (1, 108059)	1
  (1, 72034)	1
  (1, 33872)	1
  (1, 50203)	1
  (1, 79767)	1
  (1, 24690)	1
  (1, 14250)	1
  :	:
  (5249, 84047)	1
  (5249, 36019)	1
  (5249, 109790)	2
  (5249, 14864)	1
  (5249, 56540)	1
  (5249, 28404)	1
  (5249, 54799)	1
  (5249, 14478)	1
  (5249, 49630)	1
  (5249, 61648)	1
  (5249, 78942)	1
  (5249, 97512)	1
  (5249, 80432)	1
  (5249, 6221)	2
  (5249, 97267)	1
  (5249, 76227)	1
  (5249, 48348)	1
  (5249, 19592)	1
  (5249, 81721)	1
  (5249, 62)	1
  (5249, 104885)	1
  (5249, 56844)	1
  (5249, 86839)	1
  (5249, 61989)	1
  (5249, 52618)	1


In [20]:


X_train
     


<15750x112739 sparse matrix of type '<class 'numpy.int64'>'
	with 269185 stored elements in Compressed Sparse Row format>

In [21]:


y_train
     


array(['Hate', 'Free', 'Hate', ..., 'Free', 'Free', 'Hate'], dtype=object)

In [22]:


X_test
     


<9000x112739 sparse matrix of type '<class 'numpy.int64'>'
	with 152524 stored elements in Compressed Sparse Row format>

In [23]:


y_test
     


array(['Free', 'Free', 'Free', ..., 'Free', 'Free', 'Free'], dtype=object)

In [24]:
from sklearn.metrics import classification_report
#model.score(X_test,y_test)
# Define hyperparameters
max_depths = [67,2,5,9,4,6,100,1000] # maximum depth of the tree ,0
min_samples_splits = [1001,89,90,34,45,65,254] # minimum number of samples required to split an internal node 2
min_samples_leaves = [890,2001,2400,67,544,560,679,2] # minimum number of samples required to be at a leaf node 
# Train the model with different hyperparameters
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        for min_samples_leaf in min_samples_leaves:
            # Define the model
            model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,criterion='gini')
            # Train the model
            model.fit(X_train, y_train)
           #train and test
            score = model.score(X_test, y_test)
           
            # Print the results
            print(f"Max depth: {max_depth}, Min samples split: {min_samples_split}, Min samples leaf: {min_samples_leaf}, Accuracy score: {score}")

Max depth: 67, Min samples split: 1001, Min samples leaf: 890, Accuracy score: 0.5265555555555556
Max depth: 67, Min samples split: 1001, Min samples leaf: 2001, Accuracy score: 0.5137777777777778
Max depth: 67, Min samples split: 1001, Min samples leaf: 2400, Accuracy score: 0.5137777777777778
Max depth: 67, Min samples split: 1001, Min samples leaf: 67, Accuracy score: 0.5642222222222222
Max depth: 67, Min samples split: 1001, Min samples leaf: 544, Accuracy score: 0.5286666666666666
Max depth: 67, Min samples split: 1001, Min samples leaf: 560, Accuracy score: 0.5286666666666666
Max depth: 67, Min samples split: 1001, Min samples leaf: 679, Accuracy score: 0.5265555555555556
Max depth: 67, Min samples split: 1001, Min samples leaf: 2, Accuracy score: 0.5721111111111111
Max depth: 67, Min samples split: 89, Min samples leaf: 890, Accuracy score: 0.5265555555555556
Max depth: 67, Min samples split: 89, Min samples leaf: 2001, Accuracy score: 0.5137777777777778
Max depth: 67, Min sampl

In [25]:
# Test the model on the testing set
y_pred = model.predict(X_test)
            # Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        Free       0.59      0.60      0.59      4471
       Free        0.00      0.00      0.00         1
        Hate       0.60      0.59      0.59      4528

    accuracy                           0.59      9000
   macro avg       0.40      0.40      0.40      9000
weighted avg       0.59      0.59      0.59      9000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

0.5941405943232023

# Predicting the outcome ***

In [48]:
#Predicting the outcome
inp =input("Enter post :                 ")
dp = cv.transform([inp]).toarray()
dt=model.predict(dp)
if dt=='Hate':
    print("//////////////////////////////")
    print("/////the post is blocked//////")
    print("//////////////////////////////")

else:
    print(inp)
   
print(dt)

Enter post :                 አዋይ አንሄካ ፋራ
//////////////////////////////
/////the post is blocked//////
//////////////////////////////
['Hate']
